# Downloading Files using HTTP Byte Range Requests with F#

## Calculating Byte Ranges using `List.unfold`

[`List.unfold`](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-listmodule.html#unfold) is used to generate a series from an initial value, stopping when a certain condition is met.

In our case:
1. the initial value is the desired byte range size
2. the stopping condition is when the next byte range start would be greater than the total file byte size, `totalBytes`.

In [ ]:
let byteRanges (byteSize: uint64) (totalBytes: uint64) : (uint64 * uint64) list =
    (0UL, byteSize)
    |> List.unfold (fun (start, finish) ->
        // Outcome 1: stopping condition
        if start >= totalBytes then
            None
        // Outcome 2: the last byte range. stop at the total # of bytes
        else if start + byteSize > totalBytes then
            Some ((start, totalBytes-1UL),
                  (finish, finish + byteSize))
        // Outcome 3: a normal byte range
        else
            Some ((start, finish-1UL),
                  (finish, finish + byteSize))
    )

let tenMegabyteRanges = byteRanges 10_000_000UL

tenMegabyteRanges 15_000_000UL

index,Item1,Item2
0,0,9999999
1,10000000,14999999


The function above accepts `byteSize` as the # of bytes to download per request and `totalBytes` is the total byte size of the file.

The above example shows the byte size ranges calculated for downloading a 15 mb file in 10 mb chunks.

Let's step through each iteration of `List.unfold` to see if a conditional holds true/false and the outcome of the iteration:

| # | `(start, finish)` | `start >= totalBytes` ? | `start + byteSize > totalBytes` ? | Outcome # |
| - | ----------------- | ---------------------   | --------------------------------- | --------- |
| 0 | `(0, 10_000_000)` | `0 >= 15_000_000 = false` | `0 + 10_000_000 > 15_000_000 = false` | 3 - `Some ((0, 9_999_999), (10_000_000, 20_000_000))` |
| 1 | `(10_000_000,  20_000_000)` | `10_000_000 >= 15_000_000 = false` | `10_000_000 + 10_000_000 > 15_000_000 = true` | 2 - `Some ((10_000_000, 14_999_999), (15_000_000, 25_000_000))` |
| 2 | `(15_000_000,  25_000_000)` | `15_000_000 >= 15_000_000 = true` | n/a | 1 - `None` |

A few things to notice:
1. When the iteration returns a `Some` value, it returns a tuple with two elements:
    * the first element is the next element added to the series: `(0, 9_999_999)` and `(10_000_000, 14_999_999)`
    * the second element is the next value of the `List.unfold` iteration
2. `List.unfold` stops when `None` is returned.

## Calculating byte ranges using a recursive function

What if `List.unfold` was replaced with a recursive function? Here's how I would write that function:

In [ ]:
let recursiveByteRanges (byteSize: uint64) (totalBytes: uint64) : (uint64 * uint64) seq =
    let rec rbr (start, finish) =
        // Outcome 1: the last byte range. stop at the total # of bytes
        if start + byteSize >= totalBytes then
            [(start, totalBytes-1UL)]
        // Outcome 2: a normal byte range
        else
            [(start, finish-1UL)] @ (rbr (finish, finish + byteSize))

    rbr (0UL, byteSize)

let tenMegabyteRanges2 = recursiveByteRanges 10_000_000UL

tenMegabyteRanges2 25_000_000UL

index,Item1,Item2
0,0,9999999
1,10000000,19999999
2,20000000,24999999


This is similar to `List.unfold` but has one less conditoinal. However, IMHO, using `List.unfold` clarifies the code author's intent to any reader / reviewer. The reader can assume:

1. A list will be generated from a single value
2. The stopping condition will return `None` and any continuing condition will return `Some`.